In [3]:
from google.colab import drive
drive.mount('/content/drive')
training_file = '/content/drive/MyDrive/all/ML/warpeace_input.txt'
raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
print(raw_text[:200])

﻿the project gutenberg ebook of war and peace, by leo tolstoy

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restric


In [5]:
all_words = raw_text.split()
unique_words = list(set(all_words))
print(f'Number of unique words: {len(unique_words)}')


Number of unique words: 40114


In [6]:
n_chars = len(raw_text)
print(f'Total characters: {n_chars}')


Total characters: 3227520


In [7]:
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print(f'Total vocabulary (unique characters): {n_vocab}')
print(chars)


Total vocabulary (unique characters): 83
['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'á', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'î', 'ï', 'ó', 'ô', 'ö', 'ú', 'ü', 'ý', 'œ', '—', '‘', '’', '“', '”', '\ufeff']


In [8]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))
print(char_to_index)


{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '=': 27, '?': 28, '[': 29, ']': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56, 'à': 57, 'á': 58, 'â': 59, 'ä': 60, 'æ': 61, 'ç': 62, 'è': 63, 'é': 64, 'ê': 65, 'ë': 66, 'í': 67, 'î': 68, 'ï': 69, 'ó': 70, 'ô': 71, 'ö': 72, 'ú': 73, 'ü': 74, 'ý': 75, 'œ': 76, '—': 77, '‘': 78, '’': 79, '“': 80, '”': 81, '\ufeff': 82}


In [9]:
import numpy as np
seq_length = 160
n_seq = int(n_chars / seq_length)


In [10]:
X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))


In [11]:
for i in range(n_seq):
  x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
  x_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    char = x_sequence[j]
    index = char_to_index[char]
    x_sequence_ohe[j][index] = 1.
  X[i] = x_sequence_ohe
  y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1] 
  y_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    try:
      char = y_sequence[j]
    except:
      # just for ignoring out of range error
      print('final sentence')
    index = char_to_index[char]
    y_sequence_ohe[j][index] = 1.
  Y[i] = y_sequence_ohe


final sentence


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses,optimizers
tf.random.set_seed(42)


In [13]:
hidden_units = 700
dropout = 0.4


In [14]:
batch_size = 100
n_epoch= 300

In [15]:
model = models.Sequential()
model.add(layers.LSTM(hidden_units, input_shape=(None, n_vocab), return_sequences=True, dropout=dropout))
model.add(layers.LSTM(hidden_units, return_sequences=True, dropout=dropout))
model.add(layers.TimeDistributed(layers.Dense(n_vocab, activation='softmax')))


In [16]:
optimizer = optimizers.RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [17]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 700)         2195200   
                                                                 
 lstm_1 (LSTM)               (None, None, 700)         3922800   
                                                                 
 time_distributed (TimeDistr  (None, None, 83)         58183     
 ibuted)                                                         
                                                                 
Total params: 6,176,183
Trainable params: 6,176,183
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping


In [19]:
file_path = "/content/drive/MyDrive/all/ML/weights/weights_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')


In [20]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')


In [21]:
def generate_text(model, gen_length, n_vocab, index_to_char):
  index = np.random.randint(n_vocab)
  y_char = [index_to_char[index]]
  X = np.zeros((1, gen_length, n_vocab))
  for i in range(gen_length):
    X[0, i, index] = 1.
    indices = np.argmax(model.predict( X[:, max(0, i - seq_length -1):i + 1, :])[0], 1)
    index = indices[-1]
    y_char.append(index_to_char[index])
  return ''.join(y_char)



In [22]:
class ResultChecker(Callback):
  def __init__(self, model, N, gen_length):
    self.model = model
    self.N = N
    self.gen_length = gen_length
    
  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.N == 0:
      result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
      print('\nMy War and Peace:\n' + result)


In [23]:
result_checker = ResultChecker(model, 10, 500)

In [ ]:
from tensorboard.plugins.hparams import api as hp
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)
HP_HIDDEN = hp.HParam('hidden_size', hp.Discrete([300, 700]))
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([10, 20]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.1, 0.4))

def train_test_model(hparams, logdir):
    model = models.Sequential()
    model.add(layers.LSTM(hidden_units, input_shape=(None, n_vocab), return_sequences=True, dropout=dropout))
    model.add(layers.LSTM(hidden_units, return_sequences=True, dropout=dropout))
    model.add(layers.TimeDistributed(layers.Dense(n_vocab, activation='softmax')))
    model.compile(loss='mean_squared_error',
    optimizer=tf.keras.optimizers.Adam(hparams[HP_LEARNING_RATE]), metrics=['mean_squared_error'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=hparams[HP_EPOCHS])
    _, mse = model.evaluate(X_test, y_test, verbose=2)
    pred = model.predict(X_test)
    r2 = r2_score(y_test, pred)
    return mse, r2

def run(hparams, logdir):
    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(hparams=[HP_HIDDEN, HP_EPOCHS, HP_LEARNING_RATE], metrics=[hp.Metric('mean_squared_error', display_name='mse'), hp.Metric('r2', display_name='r2')],)
    mse, r2 = train_test_model(hparams, logdir)
    tf.summary.scalar('mean_squared_error', mse, step=1)
    tf.summary.scalar('r2', r2, step=1)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
session_num = 0
for hidden in HP_HIDDEN.domain.values:
    for epochs in HP_EPOCHS.domain.values:
        for learning_rate in tf.linspace(HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value, 5):
            hparams = {HP_HIDDEN: hidden, HP_EPOCHS: epochs, HP_LEARNING_RATE: float("%.2f"%float(learning_rate)), }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run(hparams, 'logs/hparam_tuning/' + run_name)
            session_num += 1

--- Starting trial: run-0
{'hidden_size': 300, 'epochs': 10, 'learning_rate': 0.1}
